In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import torchvision.transforms.v2 as v2
from torchvision.transforms import transforms
import os 
import sys
import glob
from scipy import fftpack, ndimage
import re
from skimage.morphology import opening, disk
from skimage import img_as_float
from torch.utils.data import DataLoader
from model import BaseUnet, BaseUnet3D
from data_processing_tools import remove_repeating_pattern
from evaluation_metrics import dice_overlap, intersection_over_union, accuracy, sensitivity, specificity
import timeit
from dataset import BrightfieldMicroscopyDataset

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: hndrkjs (hndrkjs-danmarks-tekniske-universitet-dtu). Use `wandb login --relogin` to force relogin


## Loading all the models

In [6]:
one_channel_unet = BaseUnet(num_inputs=1)
one_channel_unet.load_state_dict(torch.load('early_stopping_modelablation_channel_base_unet_1.pth', map_location=torch.device('cpu')))

two_channel_unet = BaseUnet(num_inputs=2)
two_channel_unet.load_state_dict(torch.load('early_stopping_modelablation_channel_base_unet_2.pth', map_location=torch.device('cpu')))

three_channel_unet = BaseUnet(num_inputs=3)
three_channel_unet.load_state_dict(torch.load('early_stopping_modelablation_channel_base_unet_3.pth', map_location=torch.device('cpu')))

four_channel_unet = BaseUnet(num_inputs=4)
four_channel_unet.load_state_dict(torch.load('early_stopping_modelablation_channel_base_unet_4.pth', map_location=torch.device('cpu')))

five_channel_unet = BaseUnet(num_inputs=5)
five_channel_unet.load_state_dict(torch.load('early_stopping_modelablation_channel_base_unet_5.pth', map_location=torch.device('cpu')))

six_channel_unet = BaseUnet(num_inputs=6)
six_channel_unet.load_state_dict(torch.load('early_stopping_modelablation_channel_base_unet_6.pth', map_location=torch.device('cpu')))

seven_channel_unet = BaseUnet(num_inputs=7)
seven_channel_unet.load_state_dict(torch.load('early_stopping_modelablation_channel_base_unet_7.pth', map_location=torch.device('cpu')))

eight_channel_unet = BaseUnet(num_inputs=8)
eight_channel_unet.load_state_dict(torch.load('early_stopping_modelablation_channel_base_unet_8.pth', map_location=torch.device('cpu')))

nine_channel_unet = BaseUnet(num_inputs=9)
nine_channel_unet.load_state_dict(torch.load('early_stopping_modelablation_channel_base_unet_9.pth', map_location=torch.device('cpu')))

ten_channel_unet = BaseUnet(num_inputs=10)
ten_channel_unet.load_state_dict(torch.load('early_stopping_modelablation_channel_base_unet_10.pth', map_location=torch.device('cpu')))

eleven_channel_unet = BaseUnet(num_inputs=11)
eleven_channel_unet.load_state_dict(torch.load('early_stopping_modelablation_channel_base_unet_11.pth', map_location=torch.device('cpu')))

base_unet_no_preprocessing = BaseUnet()
base_unet_no_preprocessing.load_state_dict(torch.load('early_stopping_modelUnet_simple_11_nopreprocess.pth', map_location=torch.device('cpu')))

base_unet_no_preprocessing_3d = BaseUnet3D()
base_unet_no_preprocessing_3d.load_state_dict(torch.load('early_stopping_modelUnet3D_simple_11_no_preprocess.pth', map_location=torch.device('cpu')))

base_unet_preprocessing = BaseUnet()
base_unet_preprocessing.load_state_dict(torch.load('early_stopping_modelUnetBase_Brightfield.pth', map_location=torch.device('cpu')))

base_unet_preprocessing_3d = BaseUnet3D()
base_unet_preprocessing_3d.load_state_dict(torch.load('early_stopping_modelUnet3D_simple_11_preprocess.pth', map_location=torch.device('cpu')))

# one_channel_unet.eval()
# two_channel_unet.eval()
# three_channel_unet.eval()
# four_channel_unet.eval()
# five_channel_unet.eval()
# six_channel_unet.eval()
# seven_channel_unet.eval()
# eight_channel_unet.eval()
# nine_channel_unet.eval()
# ten_channel_unet.eval()
# eleven_channel_unet.eval()
base_unet_no_preprocessing.eval()
base_unet_no_preprocessing_3d.eval()
base_unet_preprocessing.eval()
base_unet_preprocessing_3d.eval()

/var/folders/tw/b1bzt3x913368xjdhnk6r2b00000gn/T/ipykernel_6690/448154293.py:35: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  base_unet_no_preprocessing.load_state_dict(tor

BaseUnet3D(
  (enc_conv0): Conv3d(11, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (downconv0): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
  (batchnorm0): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (enc_conv1): Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (downconv1): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
  (batchnorm1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (enc_conv2): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (downconv2): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
  (batchnorm2): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (enc_conv3): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (downconv3): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(2, 2, 2),

## Dataloader and Testing function

In [3]:
def get_dataloader_test(sample_size=512, batch_size=1, channels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]):

    # image_root = '/zhome/70/5/14854/nobackup/deeplearningf24/forcebiology/data/brightfield'
    # mask_root = '/zhome/70/5/14854/nobackup/deeplearningf24/forcebiology/data/masks'
    image_root = 'data/brightfield'
    mask_root = 'data/masks'

    transform_test = v2.Compose([
        v2.Resize((sample_size, sample_size)),
        v2.ToDtype(torch.float32, scale=True),
        v2.ToTensor(),
    ])

    brightfield_test_datatset = BrightfieldMicroscopyDataset(root_dir_images=image_root, root_dir_labels=mask_root, train=False, validation=False, transform=transform_test, channels_to_use=channels)

    brightfield_loader_test = DataLoader(brightfield_test_datatset,  batch_size=batch_size, shuffle=False)

    return brightfield_loader_test

In [4]:
def test_model(model, test_loader, criterion, device, preprocessing=False, test_3d=False):
    test_loss = []
    dice = []
    iou = []
    acc = []
    sens = []
    spec = []

    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            # remove repeating pattern
            if preprocessing:
                for i in range(images.shape[0]):
                    images[i] = torch.tensor(remove_repeating_pattern(images[i].numpy()))

            images, labels = images.to(device), labels.to(device)

            if test_3d:
                images = images.unsqueeze(2)
                labels = labels.unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels.float())
            test_loss.append(loss.item())

            Y_pred = (outputs > 0.45).float()
            
            dice.append(dice_overlap(Y_pred, labels))
            iou.append(intersection_over_union(Y_pred, labels))
            acc.append(accuracy(Y_pred, labels))
            sens.append(sensitivity(Y_pred, labels))
            spec.append(specificity(Y_pred, labels))

    return test_loss, dice, iou, acc, sens, spec
        

## Testing the 2D and 3D Unets with and without preprocessing

In [7]:
test_loss_3d_no_preprocess, dice_3d_no_preprocess, iou_3d_no_preprocess, acc_3d_no_preprocess, sens_3d_no_preprocess, spec_3d_no_preprocess = test_model(base_unet_no_preprocessing_3d, get_dataloader_test(channels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]), nn.BCEWithLogitsLoss(), torch.device('cpu'), preprocessing=False, test_3d=True)

test_loss_3d_preprocess, dice_3d_preprocess, iou_3d_preprocess, acc_3d_preprocess, sens_3d_preprocess, spec_3d_preprocess = test_model(base_unet_preprocessing_3d, get_dataloader_test(channels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]), nn.BCEWithLogitsLoss(), torch.device('cpu'), preprocessing=True, test_3d=True)

tess_loss_2d_no_preprocess, dice_2d_no_preprocess, iou_2d_no_preprocess, acc_2d_no_preprocess, sens_2d_no_preprocess, spec_2d_no_preprocess = test_model(base_unet_no_preprocessing, get_dataloader_test(channels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]), nn.BCEWithLogitsLoss(), torch.device('cpu'), preprocessing=False, test_3d=False)

test_loss_2d_preprocess, dice_2d_preprocess, iou_2d_preprocess, acc_2d_preprocess, sens_2d_preprocess, spec_2d_preprocess = test_model(base_unet_preprocessing, get_dataloader_test(channels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]), nn.BCEWithLogitsLoss(), torch.device('cpu'), preprocessing=True, test_3d=False)

print('3D no preprocess')
print('Test loss: ', np.mean(test_loss_3d_no_preprocess))
print('Dice: ', np.mean(dice_3d_no_preprocess))
print('IoU: ', np.mean(iou_3d_no_preprocess))
print('Accuracy: ', np.mean(acc_3d_no_preprocess))
print('Sensitivity: ', np.mean(sens_3d_no_preprocess))
print('Specificity: ', np.mean(spec_3d_no_preprocess))

print('3D preprocess')
print('Test loss: ', np.mean(test_loss_3d_preprocess))
print('Dice: ', np.mean(dice_3d_preprocess))
print('IoU: ', np.mean(iou_3d_preprocess))
print('Accuracy: ', np.mean(acc_3d_preprocess))
print('Sensitivity: ', np.mean(sens_3d_preprocess))
print('Specificity: ', np.mean(spec_3d_preprocess))

print('2D no preprocess')
print('Test loss: ', np.mean(tess_loss_2d_no_preprocess))
print('Dice: ', np.mean(dice_2d_no_preprocess))
print('IoU: ', np.mean(iou_2d_no_preprocess))
print('Accuracy: ', np.mean(acc_2d_no_preprocess))
print('Sensitivity: ', np.mean(sens_2d_no_preprocess))
print('Specificity: ', np.mean(spec_2d_no_preprocess))

print('2D preprocess')
print('Test loss: ', np.mean(test_loss_2d_preprocess))
print('Dice: ', np.mean(dice_2d_preprocess))
print('IoU: ', np.mean(iou_2d_preprocess))
print('Accuracy: ', np.mean(acc_2d_preprocess))
print('Sensitivity: ', np.mean(sens_2d_preprocess))
print('Specificity: ', np.mean(spec_2d_preprocess))

3D no preprocess
Test loss:  0.32766258848064084
Dice:  0.66471165
IoU:  0.5213731
Accuracy:  0.9010626
Sensitivity:  0.5811094
Specificity:  0.9794943
3D preprocess
Test loss:  0.303239405885631
Dice:  0.7123706
IoU:  0.5669817
Accuracy:  0.9005842
Sensitivity:  0.68898076
Specificity:  0.9514856
2D no preprocess
Test loss:  0.2897508324069135
Dice:  0.6941363
IoU:  0.56036824
Accuracy:  0.9063084
Sensitivity:  0.64471835
Specificity:  0.9740524
2D preprocess
Test loss:  1.7176370281799167
Dice:  0.12770145
IoU:  0.07044108
Accuracy:  0.8165396
Sensitivity:  0.082212515
Specificity:  0.97815675


## Testing whether results are statisitcally significant

### 2D Unet vs 3D Unet

In [11]:
import numpy as np
from scipy.stats import wilcoxon
from scipy.stats import ttest_rel


modelA_scores_dice = np.array(dice_3d_no_preprocess)
modelB_scores_dice = np.array(dice_2d_no_preprocess)

modelA_scores_iou = np.array(iou_3d_no_preprocess)
modelB_scores_iou = np.array(iou_2d_no_preprocess)


# Perform a Wilcoxon signed-rank test (non-parametric)
stat_dice, p_value_dice = wilcoxon(modelA_scores_dice, modelB_scores_dice)
stat_iou, p_value_iou = wilcoxon(modelA_scores_iou, modelB_scores_iou)

print("Wilcoxon signed-rank test results for dice:")
print(f"Statistic: {stat_dice:.4f}, p-value: {p_value_dice:.4f}")

print("Wilcoxon signed-rank test results for iou:")
print(f"Statistic: {stat_iou:.4f}, p-value: {p_value_iou:.4f}")


Wilcoxon signed-rank test results for dice:
Statistic: 290.0000, p-value: 0.0005
Wilcoxon signed-rank test results for iou:
Statistic: 268.0000, p-value: 0.0002


### 3D Unet perprocessing vs preprocessing

In [12]:
modelA_scores_dice = np.array(dice_3d_no_preprocess)
modelB_scores_dice = np.array(dice_3d_preprocess)

modelA_scores_iou = np.array(iou_3d_no_preprocess)
modelB_scores_iou = np.array(iou_3d_preprocess)


# Perform a Wilcoxon signed-rank test (non-parametric)
stat_dice, p_value_dice = wilcoxon(modelA_scores_dice, modelB_scores_dice)
stat_iou, p_value_iou = wilcoxon(modelA_scores_iou, modelB_scores_iou)

print("Wilcoxon signed-rank test results for dice:")
print(f"Statistic: {stat_dice:.4f}, p-value: {p_value_dice:.4f}")

print("Wilcoxon signed-rank test results for iou:")
print(f"Statistic: {stat_iou:.4f}, p-value: {p_value_iou:.4f}")

Wilcoxon signed-rank test results for dice:
Statistic: 270.0000, p-value: 0.0002
Wilcoxon signed-rank test results for iou:
Statistic: 276.0000, p-value: 0.0003


### 2D Unet preprocessing vs no preprocessing

In [13]:
modelA_scores_dice = np.array(dice_2d_no_preprocess)
modelB_scores_dice = np.array(dice_2d_preprocess)

modelA_scores_iou = np.array(iou_2d_no_preprocess)
modelB_scores_iou = np.array(iou_2d_preprocess)


# Perform a Wilcoxon signed-rank test (non-parametric)
stat_dice, p_value_dice = wilcoxon(modelA_scores_dice, modelB_scores_dice)
stat_iou, p_value_iou = wilcoxon(modelA_scores_iou, modelB_scores_iou)

print("Wilcoxon signed-rank test results for dice:")
print(f"Statistic: {stat_dice:.4f}, p-value: {p_value_dice:.4f}")

print("Wilcoxon signed-rank test results for iou:")
print(f"Statistic: {stat_iou:.4f}, p-value: {p_value_iou:.4f}")

Wilcoxon signed-rank test results for dice:
Statistic: 422.0000, p-value: 0.0239
Wilcoxon signed-rank test results for iou:
Statistic: 362.0000, p-value: 0.0048


### 2D vs 3D preprocess

In [16]:
modelA_scores_dice = np.array(dice_3d_preprocess)
modelB_scores_dice = np.array(dice_2d_preprocess)

modelA_scores_iou = np.array(iou_3d_preprocess)
modelB_scores_iou = np.array(iou_2d_preprocess)


# Perform a Wilcoxon signed-rank test (non-parametric)
stat_dice, p_value_dice = wilcoxon(modelA_scores_dice, modelB_scores_dice)
stat_iou, p_value_iou = wilcoxon(modelA_scores_iou, modelB_scores_iou)

print("Wilcoxon signed-rank test results for dice:")
print(f"Statistic: {stat_dice:.4f}, p-value: {p_value_dice:.10f}")

print("Wilcoxon signed-rank test results for iou:")
print(f"Statistic: {stat_iou:.4f}, p-value: {p_value_iou:.10f}")

Wilcoxon signed-rank test results for dice:
Statistic: 83.0000, p-value: 0.0000000543
Wilcoxon signed-rank test results for iou:
Statistic: 75.0000, p-value: 0.0000000356
